In [13]:
import urllib
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import regex as re
from tqdm.notebook import tqdm
import os

site = 'http://www.lada-granta.net/'
folder_names = [
    ['0', 'Отзывы', 'f=139', 'LADA GRANTA'],
    ['1', 'Двигатель', 'f=22', 'LADA GRANTA'],
    ['2', 'Трансмиссия', 'f=23', 'LADA GRANTA'],
    ['3', 'Ходовая', 'f=24', 'LADA GRANTA'],
    ['4', 'Колеса', 'f=167', 'LADA GRANTA'],
    ['5', 'Электрооборудование', 'f=25', 'LADA GRANTA'],
    ['6', 'Мультимедиа', 'f=202', 'LADA GRANTA'],
    ['7', 'Кузов', 'f=26', 'LADA GRANTA'],
    ['8', 'Жидкости', 'f=27', 'LADA GRANTA'],
    ['9', 'Сервис', 'f=20', 'LADA GRANTA'],
    ['10', 'Эксплуатация', 'f=21', 'LADA GRANTA'],
    ['11', 'Тюнинг', 'f=28', 'LADA GRANTA'],
    ['12', 'Электрооборудование-Омыватели', 'f=184', 'LADA GRANTA'],
    ['13', 'Электрооборудование-Зажигание', 'f=198', 'LADA GRANTA'],
    ['14', 'Электрооборудование-Датчики', 'f=199', 'LADA GRANTA'],
    ['15', 'Электрооборудование-Проводка', 'f=200', 'LADA GRANTA'],
    ['16', 'Электрооборудование-Обогрев', 'f=201', 'LADA GRANTA'],
    ['17', 'Электрооборудование-Фары', 'f=185', 'LADA GRANTA'],
    ['18', 'Двигатель-Зажигание', 'f=195', 'LADA GRANTA'],
    ['19', 'Двигатель-Выпуск', 'f=194', 'LADA GRANTA'],
    ['20', 'Двигатель-Зажигание', 'f=195', 'LADA GRANTA'],
    ['21', 'Двигатель-Охлаждение', 'f=193', 'LADA GRANTA'],
    ['22', 'Двигатель-ГРМ', 'f=197', 'LADA GRANTA'],
    ['24', 'Двигатель-Смазка', 'f=196', 'LADA GRANTA'],
    ['25', 'Трансмиссия-МКПП', 'f=168', 'LADA GRANTA'],
    ['26', 'Трансмиссия-АКПП', 'f=169', 'LADA GRANTA'],
    ['27', 'Трансмиссия-АМТ', 'f=170', 'LADA GRANTA'],
    ['28', 'Ходовая-Подвеска', 'f=166', 'LADA GRANTA'],
    ['29', 'Ходовая-Тормозная', 'f=165', 'LADA GRANTA'],
    ['30', 'Ходовая-Рулевая', 'f=164', 'LADA GRANTA'],
    ['31', 'Кузов-Общее', 'f=176', 'LADA GRANTA'],
    ['32', 'Кузов-Отопление', 'f=174', 'LADA GRANTA'],
    ['33', 'Кузов-Кузов', 'f=175', 'LADA GRANTA'],
    ['34', 'Кузов-Салон', 'f=173', 'LADA GRANTA'],
    ['35', 'Жидкости-Топливо', 'f=182', 'LADA GRANTA'],
    ['36', 'Жидкости-Технические', 'f=180', 'LADA GRANTA'],
    ['37', 'Жидкости-ТрМасла', 'f=179', 'LADA GRANTA'],
    ['38', 'Жидкости-Смазки', 'f=177', 'LADA GRANTA'],
    ['39', 'Жидкости-МоМасла', 'f=178', 'LADA GRANTA'],
    ['40', 'Жидкости-Присадки', 'f=181', 'LADA GRANTA'],
    ['41', 'Жидкости-Автохимия', 'f=183', 'LADA GRANTA'],
    ['42', 'Тюнинг-Стайлинг', 'f=142', 'LADA GRANTA'],
    ['43', 'Тюнинг-Тюнинг', 'f=143', 'LADA GRANTA'],
    ['44', 'Тюнинг-Противоугон', 'f=141', 'LADA GRANTA'],
    ['45', 'Тюнинг-Автозвук', 'f=140', 'LADA GRANTA']
]

На странице раздела у нас две задачи:  
1. Найти и сохранить все ссылки на темы
2. Найти ссылку на следующую страницу раздела или убедиться, что это последняя страница раздела.

In [14]:
def collect_themes(site, folder):
    
    site_address = site + 'forumdisplay.php?' + folder
    themes = []
    flag = True

    while flag:
    
        resp = urlopen(site_address) # скачиваем файл
        html = resp.read().decode('utf-8') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
        flag = False
        for link in soup.find_all('a'):
            if link.has_attr('href') and link.has_attr('id'):
                if re.search('thread_title_', str(link.get('id'))):
                    s = link.get('href')
                    s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                    themes.append([link.get_text(), s])
            if not flag and link.has_attr('href') and link.has_attr('rel'):
                if link.get('rel')[0] == 'next':
                    flag = True
                    s = link.get('href')
                    site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]
    return themes

На странице темы у нас три задачи:  
1. Понять, что являеся шапкой темы (это сообщение #1), прочитать ее только один раз - НЕ СДЕЛАНО
2. Прочитать и сохранить все сообщения (кроме шапки) на странице темы
3. Найти ссылку на следующую страницу темы или убедиться, что это последняя страница темы.

In [15]:
def collect_messages(site, themes):

    messages = []
    for theme in tqdm(themes):
        
        print(theme[0])
        theme_address = theme[1]
        thread = re.search('t=[0-9]+', theme_address)[0]
        flag = True
        theme_messages = []
        post_data = {}
        
        while flag:
            resp = urlopen(theme_address) # скачиваем файл
            html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
            soup = BeautifulSoup(html, 'html.parser') # делаем суп
            #Собираем некоторые метаданные сообщений на текущей странице
            tables = soup.find_all('table')
            for next_table in tables:
                if next_table.has_attr('id') and re.search('post[0-9]+', next_table['id']):
                    post_column = next_table.find('td')
                    post_date = post_column.get_text().strip()
                    post_data[re.sub('post', '', next_table['id'])] = [post_date]
            #Собираем сами сообщения
            divs = soup.find_all('div')
            for div in divs:
                if div.has_attr('id') and re.search('(postmenu_)([0-9]+)(_menu)', div['id']):
                    continue
                if div.has_attr('id') and re.search('postmenu_', div['id']):
                    author = div.find('a')
                    if author != None:
                        author_id = re.search('u=[0-9]+', author['href'])[0]
                        author_nickname = author.get_text()
                    else:
                        author_id = div.get_text().strip()
                        author_nickname = 'Guest'
                    post_data[re.sub('postmenu_', '', div['id'])].append([author_id, author_nickname])
                if div.has_attr('id') and re.search('post_message_', div['id']):
                    message_id = re.sub('post_message_', '', div['id'])
                    if message_id in post_data:
                        post_date = post_data[message_id][0]
                        author = post_data[message_id][1][1]
                    else:
                        post_date = ''
                        author = ''
                    for item in div.children:
                        if item.name == 'div':
                            item.clear()
                    s = re.sub('[\']+', '`', div.get_text())
                    s = '\'' + re.sub('[\s]+', ' ', s).strip() + '\''
                    if len(s) > 2:
                        theme_messages.append([post_date, author, s])
            # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
            # Если ссылка найдена, 
            flag = False 
            for link in soup.find_all('a'):
                if not flag and link.has_attr('href') and link.has_attr('rel'):
                    if link.get('rel')[0] == 'next':
                        flag = True
                        s = link.get('href')
                        theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
        messages.append(['\''+ theme[0] + '\'', theme_messages])
    return messages

In [16]:
def save_messages(messages, folder_name):
    f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
    with open(f_name, 'w', encoding='utf-8') as ouf:
        ouf.write('Code,Folder,Theme,Date,Time,Author,Message\n')
        for message in messages:
            header = folder_name[:2] + [message[0]]
            for item in message[1]:
                ouf.write(','.join(header + item))
                ouf.write('\n')

In [17]:
def scraper(site, folder_names):
    for folder_name in folder_names:
        if len(folder_name[2]) > 0:
            folder = folder_name[2]
            current_themes = collect_themes(site, folder)
            current_messages = collect_messages(site, current_themes)
            save_messages(current_messages, folder_name)
    print('Information collected')

In [18]:
%load_ext ipytelegram
%reload_ext ipytelegram
import telepot
bot = telepot.Bot('1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A')
response = bot.getUpdates()
%telegram_setup 1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A 1305740495

The ipytelegram extension is already loaded. To reload it, use:
  %reload_ext ipytelegram


In [19]:
%%telegram_send Next message
scraper(site, folder_names)
'Information collected'

Ваше мнение о Лада Гранта - 5
Авто лучше гранты за эти деньги
Ваше мнение о Лада Гранта - 4
Друзья!!! Давайте поделимся, кто сколько проехал, и что произошло за этот период-4
Lada Granta на вторичном рынке
Статистика по комплектациям
Моя гранта лифтбек стандарт + дополнения
Дальние поездки
Ваше мнение о Лада Гранта - Архив
Выбор Гранты!
Отзывы владельцев Лада-Гранта о деятельности официальных дилеров Автоваза
Друзья!!! Давайте поделимся, кто сколько проехал, и что произошло за этот период - Архив
Годовая стоимость владения Грантой
Залил Гранту лужей, помогите пожалуйста.
Моя Гранта, Ростов-на-Дону, 3000 км
Статистика поломок
Гранта люкс 2013 отзыв владельца.
Маленький отчетец.
Только положительные отзывы
Расходы и не только. Лада-Гранта.
Доходы Грантоводов
Неожиданная встреча!!!
Фотографии Я и моя Лада Гранта.
лада Гранта лифтбек 2015.
Липовые отзывы о Гранте
Моя Гранта!
Для таксистов
Кто ждет или получил цвет ПЕРСЕЙ.
С иномарки на LADA GRANTA-3
Как я гранту чинил
Гранта спасла меня
Пр

Линейка двигателей для разных комплектаций  Лады Гранты
Рекомендуется к прочтению всем пользователям!
Все проблемы пуска (холодный, долгий и т.д.)
Проблемы ХХ (глохнет, нестабильность и пр.)
Пропуски воспламенения
Звуки работы (вой, свист, стуки и т.д.)
Цокот из двигателя непонятно откуда, нашли
Двигатель 21129
Двигатель: формат вопрос-ответ - 2
Ремонт двигателя после обрыва ремня ГРМ (21116, 11186, 21126, 21127) - 2
Проблема с движком - 3
Подъём клапана на ДВС 21186
Очистители двигателя, инжектора, клапанов
При каких оборотах двигателя вы ездите?
Компрессия ДВС 21116
Двигатель 11186
2110 поршневая в 21116 мотор
Переход с 8V => 16V
Обкатка автомобиля
Определение степени износа мотора по продолжительности стука только что запущенного холодного двигателя.
Самый беспроблемный двигатель
Увеличение степени сжатия двигателя 11183
Отличительные особености двигателя 21116
Ремонт двигателя после обрыва ремня ГРМ (21116, 11186, 21126, 21127) - Архив
Проблема с движком - Архив
Двигатель: формат в

Ваш выбор: МКПП или АКПП или АМТ?
Стук при движении на малой скорости
Захрустели шрусы
Замена АКПП на МКПП
Замена пыльника ШРУСа
Хомуты пыльников, нужен совет
Дрожь при троганье
Блокировка дифференциала
4WD c АКПП на гранту!?
Свист. Откуда?..
АКП или МКП ? -2
привод
LADA с полуавтоматом
странный писк из под капота
Не сток коробка на гранту
АКП или МКП ? - Архив



Помогите определить причину вибрации (руля, педалей, кузова)
Помогите определить причину звуков (стуки, щелчки, писки и т.д.)
шаровые опоры
[Опрос] Выбор тормозов.
Машину тянет в сторону....
Удар задней балкой об асфальт.
Можно ли поднимать домкратом за балку?
Дополнительные развальные пластины
Расширение колеи
Ходовая, тормоза и управление Лада Гранта
Разное.... Архив.



Диски на Лада Гранта - 3
Давление в шинах
Колесные болты для штамповок и литья (ковки)
Выбор зимних шин - 3
Самостоятельная замена колёс
Выбор зимних шин - 4
Большие колеса на Гранту - 2
Шины для Гранты - 5
Покупать второй комплект дисков или перебортовывать?
Шины для Гранты-Архив
Выбор размера колеса: 13" или 14"
Опыт эксплуатации штатных шин "Кама -217"
Обкатка новых шин.
Выбор зимних шин - Архив
Лучшие шины для рыбалки леса
Секретки на колеса
Большие колеса на Гранту - Архив
Выбор компрессора для накачивания шин
Уводит машину после смены резины
Нагрев колёсных дисков
Диски на Лада Гранта - Архив
Сколько стоит заклеить заплаткой прокол в шине?
Заглушки на заводских литых дисках
Стоковые диски на норме? Кто производитель и какая ширина обода?
Пропала динамика
Как считаете, стоит ли доплачивать за литьё +4 р.



...

ЛКП авто (полироли, уход и др.средства)
Полироль для фар



Накладка на задний бампер, молдинг багажника и нижний спойлер переднего бампера от YUAGO
Парктроник в Гранте. Не нужная опция или полезная примочка?
Зеркало заднего вида
Перетяжка руля
Защита картера
Установка прицепного устройства на Ладу Гранту
Рейлинги на Лада Гранта
Подлокотник
Задняя спинка с лыжницей ? Есть ли варианты для седана?
Датчик дождя и света ????
Дефлекторы вставные
Стеклоподьёмники без зажигания
Аэрография для LADA GRANTA
Бардачок на месте подушки безопасности пассажира
Подушка безопасности пассажира
Молдинги на двери Гранты
TPMS система контроя давления в шинах
Мультируль для Лада Гранта
Обвесы на ЛАДА ГРАНТА!!! - 2
Решетка радиатора.
Рамка номера из нержавейки.
Бардачёк доработка
Тонировочная полоса на лобовое стекло.
Бокс на крышу для седанов и (хетчбеков)лифтбеков от YUAGO
Атермальные пленки на Гранте.
Спойлер и защита картера!!!
Накладки на педали
Дефлектор капота
Пороги.
"Морда" от калина 2 на гранту
Реснички на фары
Растяжка передних стоек
Рейлинги или съемный б

Заводские прошвки ЭБУД
Чип тюнинг - 2
Доп свет.
Лада Гранта - Пол из экокожы
Установка промежуточного вала
Доводчик стёкол
Тюнинг: распредвалы
Доводчики-ограничители дверей
"ТАНКЕТКА" из Гранты с АБС - возможно?
Преобразователь 12 - 220В и использование автомобиля как генератора электроэнергии
Вылет 25, ширина 6,  195/60 R14, настройка сход-развал...
Панель приборов Gamma GF 851
Чудо техники :shock:
Чип тюнинг - Архив
Установка турбины на Гранту
Шайтан-труба
Холодный впуск
Штучки от иномарки на Гранту.
тюнинг замка капота
Корвет М11.ET



Установка сигнализации на Гранту - 3
Где обычно ставят кнопку Валет?
Пробка бензобака - защита от пионэров
Установка сигнализации на Гранту - 2
Помогите пожалуйста выбрать сигнализацию
Проблема со сторонней сигнализацией
Оптимальная сигнализация для Гранта "Норма"-2
Сигналка на FL 2019 классик-оптима.
Штатная противоугонная система
Блокиратор руля
Сигнализации на Гранту люкс
Шерханы вскрываются
Модуль comfortlin v2 new
Включение зажигания, стартера с кнопки
Установка Pantera CL-550
Механическая защита
Нестандартная защита от угона
ключи остались в машине.
Где обычно прячут обходчик иммо?
ВНИМАТЕЛЬНО С КОНЦЕВИКАМИ ОТ ПРИОРЫ
Интервью с угонщиком
открытие багажника с брелка сигнализации
Сигнализация Tomahawk Z5
Внимание все владельцам Старлайн А93!!!
Ваше мнение о ULTRA БЛ-ЭК
Starline a91 не мигает одна сторона поворотников
Проблема с сигналкой Старлайн А91 Диалог
А что можно сказать о сигналках KGB?
Установка Сигнализацию на Гранту
Охранно-поисковая систем АвтоФон
Оптимальная сигнализаци

Провод под бардочком
Замена штатной акустики в комплектации "Люкс"
Обсуждаем 2-дин магнитолы и их установку
Пропал звук сразу во всех колонках
Музыка в Гранте-Стандарт.
Штатная магнитола заикается
Замок зажигания и АСС
Аудиосистемы (магнитола, динамики, антенна). Кто что поставил.
Соединие по bluetooth
Размер динамиков в Гранте???
Камера заднего вида
рамка-переходник 2-DIN
Антенны
Проблема с магнитолой
Нет контакта на Remote
Акустические подиумы LADA Granta
Подрулевой джойстик от FORD для Sony Pioneer
Нужен совет.
Установка задней полки
Музыка с телефона фонит... Помогите.
Динамики в задних дверях
Музыка в Ладе Гранте -3
Установка ГУ и колонок на 16,5 см
ПИТАНИЕ УСИЛИТЕЛЯ(СИЛОВОЙ КАБЕЛЬ)
Музыка в Ладе Гранте - Архив
Часы на штатной магнитоле
Задняя акустическая полка на ЛИФТБЕК
компактный subwoofer
Штатная антенна гранта люкс
+12 для антенного усилителя
Подбор оптимального сабвуфера и усилителя
Куда установить усилитель?
Вопрос по установке усилителя, как провести провод плюсовой
Устан

In [ ]:
import pandas as pd
mess = pd.read_csv('./LADA VESTA/LADA VESTA_10_Эксплуатация.csv', quotechar="'")
mess

In [10]:
folder_names = [['9', 'Сервис', 'f=20', 'LADA GRANTA']]
folder = folder_names[0][2]
current_themes = collect_themes(site, folder)

http://www.lada-granta.netforumdisplay.php?f=20


URLError: <urlopen error [Errno 11004] getaddrinfo failed>

In [ ]:
themes = []
flag = True

while flag:
    
    resp = urlopen(site_address) # скачиваем файл
    html = resp.read().decode('utf-8') # считываем содержимое
    soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
    flag = False
    for link in soup.find_all('a'):
        if link.has_attr('href') and link.has_attr('id'):
            if re.search('thread_title_', str(link.get('id'))):
                s = link.get('href')
                s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                themes.append([link.get_text(), s])
        if not flag and link.has_attr('href') and link.has_attr('rel'):
            if link.get('rel')[0] == 'next':
                flag = True
                s = link.get('href')
                site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]

In [99]:
themes = [['Механика, робот или автомат?',
  'http://www.lada-vesta.net/showthread.php?t=47']]

In [ ]:
messages = []
for theme in tqdm(themes):
    theme_address = theme[1]
    thread = re.search('t=[0-9]+', theme_address)[0]
    flag = True
    theme_messages = []
    post_data = {}
    while flag:
        resp = urlopen(theme_address) # скачиваем файл
        html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
        #Собираем некоторые метаданные сообщений на текущей странице
        tables = soup.find_all('table')
        for next_table in tables:
            if next_table.has_attr('id') and re.search('post[0-9]+', next_table['id']):
                post_column = next_table.find('td')
                post_date = post_column.get_text().strip()
                post_data[re.sub('post', '', next_table['id'])] = [post_date]
        #Собираем сами сообщения
        divs = soup.find_all('div')
        for div in divs:
            if div.has_attr('id') and re.search('(postmenu_)([0-9]+)(_menu)', div['id']):
                continue
            if div.has_attr('id') and re.search('postmenu_', div['id']):
                author = div.find('a')
                if author != None:
                    author_id = re.search('u=[0-9]+', author['href'])[0]
                    author_nickname = author.get_text()
                else:
                    author_id = div.get_text().strip()
                    author_nickname = 'Guest'
                post_data[re.sub('postmenu_', '', div['id'])].append([author_id, author_nickname])
                print(re.sub('postmenu_', '', div['id']), post_data[re.sub('postmenu_', '', div['id'])])
            if div.has_attr('id') and re.search('post_message_', div['id']):
                message_id = re.sub('post_message_', '', div['id'])
                if message_id in post_data:
                    post_date = post_data[message_id][0]
                    author = post_data[message_id][1][1]
                else:
                    post_date = ''
                    author = ''
                for item in div.children:
                    if item.name == 'div':
                        item.clear()
                s = re.sub('[\']+', '`', div.get_text())
                s = '\'' + re.sub('[\s]+', ' ', s) + '\''
                if len(s) > 0:
                    theme_messages.append([post_date, author, s])
        # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
        # Если ссылка найдена, 
        flag = False 
        for link in soup.find_all('a'):
            if not flag and link.has_attr('href') and link.has_attr('rel'):
                if link.get('rel')[0] == 'next':
                    flag = True
                    s = link.get('href')
                    theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
    messages.append(['\''+ theme[0] + '\'', theme_messages])

In [ ]:
messages

In [85]:
f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
with open(f_name, 'w', encoding='utf-8') as ouf:
    ouf.write('Code,Folder,Theme,Date,Author,Message\n')
    for message in messages:
        header = folder_name[:2] + [message[0]]
        for item in message[1]:
            ouf.write(','.join(header + item))
            ouf.write('\n')

In [69]:
folder_name = ['0', 'Отзывы', 'f=25', 'LADA VESTA']
f_name = folder_name[3] + '/' + folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv'
f_name

'LADA VESTA/LADA VESTA_0_Отзывы.csv'

In [ ]:
'''messages = []
theme_address = 'http://www.lada-vesta.net/showthread.php?t=9571'
resp = urlopen(theme_address) # скачиваем файл
html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
soup = BeautifulSoup(html, 'html.parser') # делаем суп
divs = soup.find_all('div')
for div in divs:
    if div.has_attr('id'):
        if re.search('post_message_', div['id']):
            for item in div.children:
                if item.name != 'div':
                    s = str(item).strip()
                    if len(s) > 0:
                        messages.append([s])
print(messages)'''